In [16]:
import pandas as pd
import math

In [17]:
files = ['_vehicle_gps_position_0','_cpuload_0','_battery_status_0','_radio_status_0','_vehicle_air_data_0','_vehicle_land_detected_0']
def getFiles(name,location):
    data = []
    for f in files:
        data.append(pd.read_csv(location+'/'+name+f+'.csv'))
    return data

In [18]:
data = getFiles('milestone3TestData','data')

In [19]:
data[0].tail()

,timestamp,time_utc_usec,lat,lon,alt,alt_ellipsoid,s_variance_m_s,c_variance_rad,eph,epv,...,vel_n_m_s,vel_e_m_s,vel_d_m_s,cog_rad,timestamp_time_relative,heading,heading_offset,fix_type,vel_ned_valid,satellites_used
1002,257880190,1552674846800316,323873505,-1170765258,13335,-21110,0.219,0.606424,0.809,1.130,...,-0.010,0.010,-0.029,4.296656,0,NaN,0.0,4,1,18
1003,258080191,1552674847000316,323873505,-1170765257,13333,-21112,0.225,0.606434,0.808,1.129,...,-0.039,-0.013,-0.049,4.296656,0,NaN,0.0,4,1,18
1004,258279440,1552674847200316,323873504,-1170765256,13314,-21130,0.232,0.606444,0.808,1.128,...,-0.005,-0.020,-0.062,4.296656,0,NaN,0.0,4,1,18
1005,258481080,1552674847400316,323873504,-1170765255,13328,-21117,0.216,0.606454,0.807,1.126,...,-0.008,0.002,0.017,4.296656,0,NaN,0.0,4,1,18
1006,258684279,1552674847600316,323873504,-1170765255,13314,-21131,0.215,0.606464,0.806,1.125,...,-0.014,-0.023,0.031,4.296656,0,NaN,0.0,4,1,18


In [111]:
def getGpsData(data, cols):
    df = pd.DataFrame()
    times = data['timestamp']
    #prevTime = -1
    for t in times:
        #currentTime = int(t/1000000)
        #if currentTime != prevTime:
            #prevTime = currentTime
        row = data.loc[data['timestamp'] == t]
        df = df.append(row[cols])
            #print (row[cols])
    return df
    

In [112]:
gps = getGpsData(data[0], ['timestamp','time_utc_usec','lat','lon','alt','noise_per_ms','jamming_indicator','vel_m_s','heading','satellites_used']).reset_index(drop=True)

In [113]:
gps.head()

,timestamp,time_utc_usec,lat,lon,alt,noise_per_ms,jamming_indicator,vel_m_s,heading,satellites_used
0,57471124,1552674646400308,323873506,-1170765251,14912,90,4,0.027,NaN,18
1,57669395,1552674646600308,323873507,-1170765251,14876,90,4,0.062,NaN,18
2,57870534,1552674646800308,323873507,-1170765251,14873,90,4,0.047,NaN,18
3,58073571,1552674647000308,323873508,-1170765250,14873,90,4,0.051,NaN,18
4,58271167,1552674647200308,323873508,-1170765250,14868,90,7,0.035,NaN,18


In [114]:
def getData(baseline, data, cols):
    df = pd.DataFrame()
    times = data['timestamp']
    baseline_times = baseline['timestamp'] #gps data
    start_index = 0
    match_index = []
    for base_time in baseline_times:
        last_diff = math.inf
        for index,t in enumerate(times[start_index:]):
            current_diff = abs(t - base_time)
            if current_diff < last_diff:
                last_diff = current_diff
                start_index += index
            else:
                break
        row = data.iloc[start_index]
        df = df.append(row[cols])
    return df

In [115]:
def getData2(baseline, data, cols):
    df = pd.DataFrame()
    times = data['timestamp']
    baseline_times = baseline['timestamp'] #gps data
    match_index = []
    for base_time in baseline_times:
        start_index = 0
        last_diff = math.inf
        for index,t in enumerate(times):
            current_diff = abs(t - base_time)
            if current_diff < last_diff:
                last_diff = current_diff
                start_index = index
            else:
                break   
        row = data.iloc[start_index]
        df = df.append(row[cols])
    return df

In [120]:
cpu = getData(gps,data[1],['load','ram_usage']).reset_index(drop=True)
battery = getData(gps,data[2], ['voltage_filtered_v','remaining','warning','discharged_mah']).reset_index(drop=True)
radio = getData(gps,data[3], ['noise','remote_noise','txbuf','rssi','remote_rssi','rxerrors','fix']).reset_index(drop=True)
air = getData2(gps,data[4],['baro_temp_celcius','baro_pressure_pa']).reset_index(drop=True)
land = getData(gps,data[5],['landed','ground_contact']).reset_index(drop=True)

df = pd.DataFrame()
df = pd.concat([df,gps,cpu,battery,radio,air,land],axis=1, sort=False)
df = df[:-2]
df['time'] = df.index

In [121]:
df

,timestamp,time_utc_usec,lat,lon,alt,noise_per_ms,jamming_indicator,vel_m_s,heading,satellites_used,...,remote_noise,remote_rssi,rssi,rxerrors,txbuf,baro_pressure_pa,baro_temp_celcius,ground_contact,landed,time
0,57471124,1552674646400308,323873506,-1170765251,14912,90,4,0.027,NaN,18,...,65.0,107.0,104.0,2.0,97.0,101701.99,25.300000,0.0,1.0,0
1,57669395,1552674646600308,323873507,-1170765251,14876,90,4,0.062,NaN,18,...,65.0,107.0,104.0,2.0,97.0,101700.99,25.300000,0.0,1.0,1
2,57870534,1552674646800308,323873507,-1170765251,14873,90,4,0.047,NaN,18,...,65.0,107.0,104.0,2.0,97.0,101698.00,25.310000,0.0,1.0,2
3,58073571,1552674647000308,323873508,-1170765250,14873,90,4,0.051,NaN,18,...,65.0,107.0,104.0,2.0,97.0,101701.99,25.310000,0.0,1.0,3
4,58271167,1552674647200308,323873508,-1170765250,14868,90,7,0.035,NaN,18,...,65.0,107.0,104.0,2.0,97.0,101703.00,25.320000,0.0,1.0,4
5,58471137,1552674647400308,323873509,-1170765250,14855,90,7,0.031,NaN,18,...,54.0,107.0,104.0,2.0,99.0,101705.00,25.330000,0.0,1.0,5
6,58669085,1552674647600308,323873509,-1170765249,14863,90,7,0.015,NaN,18,...,54.0,107.0,104.0,2.0,99.0,101703.00,25.340000,0.0,1.0,6
7,58875524,1552674647800308,323873509,-1170765249,14849,90,7,0.017,NaN,18,...,54.0,107.0,104.0,2.0,99.0,101699.00,25.350000,0.0,1.0,7
8,59077130,1552674648000308,323873509,-1170765250,14856,90,7,0.012,NaN,18,...,54.0,107.0,104.0,2.0,99.0,101703.00,25.350000,0.0,1.0,8
9,59276144,1552674648200308,323873510,-1170765249,14866,91,4,0.058,NaN,18,...,55.0,107.0,104.0,2.0,97.0,101699.00,25.359999,0.0,1.0,9


In [122]:
df.to_csv('milestone3_modified.csv')